In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms

print(f"{torch.__version__}, {torchvision.__version__}")

2.0.1+cu118, 0.15.2+cu118


In [2]:
# device agnostic
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# import torchinfo
try:
    from torchinfo import summary
except:
    print(f"[INFO] installing torchinfo...")
    !pip install torchinfo
    from torchinfo import summary

In [4]:
# create function to check model summary
def check_model_summary(model: torch.nn.Module,
                       input_size = [32, 3, 224, 224],
                       col_names = ["input_size", "output_size", "num_params", "trainable"],
                       col_width = 20):
    return summary(
        model = model,
        input_size = input_size,
        col_names = col_names,
        col_width = col_width
    )

In [5]:
# import my modules
try:
    from sources import datasetup, engine, utils
except:
    !git clone git@github.com:buupq/torch.git
    !mv torch/sources .
    !rm -rf torch
    from sources import datasetup, engine, utils

Cloning into 'torch'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 5), reused 16 (delta 2), pack-reused 0
Receiving objects: 100% (27/27), 19.03 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (5/5), done.
mv: cannot move 'torch/sources' to './sources': Directory not empty


AttributeError: module 'torch.utils' has no attribute 'tensorboard'

In [ ]:
# download data
image_path = datasetup.download_data(source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
image_path

In [ ]:
# setup train and test directories
train_dir = image_path / "train"
test_dir = image_path / "test"
train_dir, test_dir

# playing with one pretrain model

In [ ]:
# from weights of a pretrained model get its transformation methods and model
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
automatic_transforms = weights.transforms()
model = torchvision.models.efficientnet_b0(weights=weights).to(device)
print(f"automatic transforms: {automatic_transforms}")

In [ ]:
# check_model_summary(model=model)

In [ ]:
# create dataloaders from the available transforms
train_dataloader, test_dataloader, class_names = datasetup.create_dataloaders(
    train_dir = train_dir,
    test_dir = test_dir,
    train_transforms=automatic_transforms,
    test_transforms=automatic_transforms,
    batch_size = 32,
    num_workers=1
)
train_dataloader, test_dataloader, class_names

In [ ]:
# free model feature layer parameters
for para in model.features.parameters():
    para.requires_grad = False

In [ ]:
# check model classifier head
model.classifier

In [ ]:
# modify model classifier head
model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(in_features=1280, out_features=3, bias=True)
).to(device)
model.classifier

In [ ]:
# check_model_summary(model=model)

In [ ]:
# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [ ]:
# train the modified model

results = engine.train(
    model = model,
    train_dataloader = train_dataloader, 
    test_dataloader = test_dataloader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    epochs=5,
    device=device
)

# use train function with results recorded to tensorboard

In [ ]:
# import Summary writer from tensorboard
from torch.utils.tensorboard.writer import SummaryWriter
writer = SummaryWriter()
writer

In [ ]:
# setup loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [7]:
# do training with results written to tensorboard writer
results = engine.train_tsb_writer(
    model=model,
    train_dataloader = train_dataloader,
    test_dataloader = test_dataloader,
    loss_fn = loss_fn,
    optimizer = optimizer,
    epochs=5,
    device=device,
    writer=writer
)

NameError: name 'engine' is not defined

In [8]:
from torch.utils.tensorboard.writer import SummaryWriter